In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import movie_reviews
# text
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer, TfidfVectorizer
# classification
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso #  модель линейной регрессии со встроенной L1-регуляризацией весов (ограничение на сумму модулей весов).
from sklearn.linear_model import Ridge # модель линейной регрессии со встроенной L2-регуляризацией весов (ограничение на сумму квадратов весов).
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import GaussianNB  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
# classterization
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, SpectralClustering, DBSCAN, MiniBatchKMeans
# preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, label_binarize, OneHotEncoder
# mertrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# embedding vectors
# Важно
# It is highly recommended to use another dimensionality reduction method (e.g. PCA for dense data or TruncatedSVD for sparse data)  
# to reduce the number of dimensions to a reasonable amount (e.g. 50) if the number of features is very high.  
# This will suppress some noise and speed up the computation of pairwise distances between samples
from sklearn.manifold import TSNE
# reduce the dimensionality
from sklearn.decomposition import PCA, TruncatedSVD

import string
import random
from sklearn import datasets
import pandas as pd
import numpy as np
from datetime import date,datetime
import matplotlib.pyplot as plt
import seaborn as sns

from pylab import rcParams
rcParams['figure.figsize'] = 8, 6

In [2]:
# Загружаем набор данных
iris_df = datasets.load_iris()

### Lable encoding

если порядок совпадает с ожидаемым, то есть по алфавиту буквы идут и подходит, что будет от 0 до n  
или нам не важно какая буква будет с какой цифорой

In [4]:
le = LabelEncoder()

l = ['a', 'c', 'b', 'd', 'e']

le.fit_transform(l)

array([0, 2, 1, 3, 4], dtype=int64)

если нам нужно правильный порядок сделать, то используем label_binarize  
label_binarize делает one hot encoding, и если нам нужно от 0 до n, то используем argmax

In [11]:
from sklearn.preprocessing import label_binarize

l = ['bad', 'excellent', 'medium', 'very good', 'very bad']

label_binarize(l, classes=['excellent', 'very good', 'medium', 'bad', 'very bad']).argmax(axis=1)

array([3, 0, 2, 1, 4], dtype=int64)

In [12]:
from sklearn.preprocessing import label_binarize

l = ['bad', 'excellent', 'medium', 'very good', 'very bad']

label_binarize(l, classes=['excellent', 'very good', 'medium', 'bad', 'very bad'])

array([[0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1]])

### Выбор признаков, которые лучше других влияют на модель 

In [15]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
X = [[ 0.87, -1.34,  0.31 ],
     [-2.79, -0.02, -0.85 ],
     [-1.34, -0.48, -2.55 ],
     [ 1.92,  1.48,  0.65 ]]
y = [0, 1, 0, 1]
selector = SelectFromModel(estimator=LogisticRegression()).fit(X, y)
selector.estimator_.coef_

array([[-0.3252302 ,  0.83462377,  0.49750423]])

Порог, по которому селектор разбивает

In [21]:
selector.threshold_

0.3333333333333333

Результат в вдие булевых значений после сравнения порого и коэффициентов

In [7]:
selector.get_support()

array([False,  True, False])

И теперь забираем только важные признаки для этой модели
По сути просто берем фичи, у которых истина после разделения по селекторву

In [8]:
selector.transform(X)

array([[-1.34],
       [-0.02],
       [-0.48],
       [ 1.48]])

In [22]:
from sklearn.tree import DecisionTreeClassifier  

selector = SelectFromModel(estimator=DecisionTreeClassifier()).fit(X, y)
dt = DecisionTreeClassifier().fit(X, y)
selector.estimator_.feature_importances_

array([0., 1., 0.])

In [23]:
selector.threshold_

0.3333333333333333

## Сокращение размерности

PCA

In [16]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
pca = PCA(n_components=2)
pca.fit(X)
PCA(n_components=2)
# Чтобы проверить соотношение дисперсии, объясненную этим СПС, мы можем рассмотреть pca.explained_variance_ratio_ :
print(pca.explained_variance_ratio_)
print(pca.singular_values_)
print()
# линейные комбинации
# Обратите внимание, что первые два компонента в каждом векторе на несколько порядков больше других, что показывает,   
# что СПС признал, что дисперсия содержится главным образом в первых двух столбцах
print(pca.components_)

[0.99244289 0.00755711]
[6.30061232 0.54980396]

[[-0.83849224 -0.54491354]
 [ 0.54491354 -0.83849224]]


In [17]:
# Загрузка данных
data = datasets.load_iris()
X, y = data.data, data.target

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применение PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Обучение SVM (C-Support Vector Classification) на данных после PCA
svm = SVC()
svm.fit(X_train_pca, y_train)

# Оценка производительности модели
accuracy = svm.score(X_test_pca, y_test)
print(f'Accuracy after PCA: {accuracy:.2f}')

Accuracy after PCA: 1.00


TruncatedSVD

In [14]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np
np.random.seed(0)
X_dense = np.random.rand(100, 100)
X_dense[:, 2 * np.arange(50)] = 0
X = csr_matrix(X_dense)
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
svd.fit(X)
TruncatedSVD(n_components=5, n_iter=7, random_state=42)
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())
print(svd.singular_values_)

[0.01570766 0.05122679 0.04998062 0.04795064 0.04539933]
0.21026503465070323
[35.24105443  4.5981613   4.54200434  4.44866153  4.32887456]


## Классификация

In [ ]:
# Пример данных
nltk.download('movie_reviews')

lemma = nltk.WordNetLemmatizer()
stop_words = set(nltk.corpus.stopwords.words('english'))
punctuation = set(string.punctuation)

# Загрузка данных
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)
# Подготовка данных
texts = [' '.join(doc) for doc, _ in documents]
labels = [1 if category == 'pos' else 0 for _, category in documents]
prep_text = []
# Токенизация и лематизация
for sentence in texts:
    sentence = sentence.lower()
    tokens = word_tokenize(sentence)
    lem_tokens = [lemma.lemmatize(token) for token in tokens]
    lem_filt_tokens = [token for token in lem_tokens if token not in stop_words and token not in punctuation]
    prep_sentence = ' '.join(lem_filt_tokens)
    prep_text.append(prep_sentence)
   
# Создание BoW модели
vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(prep_text)

# Разделение данных на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(bow, labels, test_size=0.3)


MultinomialNB

In [ ]:
# Обучение классификатора
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Оценка классификатора
predictions = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

print(classification_report(y_test, predictions, ))

Accuracy: 0.8033333333333333
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       304
           1       0.81      0.79      0.80       296

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600



LogisticRegression

In [ ]:
# Обучение классификатора
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

# Оценка классификатора
predictions = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

print(classification_report(y_test, predictions, ))

Accuracy: 0.835
              precision    recall  f1-score   support

           0       0.83      0.85      0.84       304
           1       0.84      0.82      0.83       296

    accuracy                           0.83       600
   macro avg       0.84      0.83      0.83       600
weighted avg       0.84      0.83      0.83       600



LinearDiscriminantAnalysis

In [ ]:
# Обучение классификатора
classifier = LinearDiscriminantAnalysis()
classifier.fit(X_train.toarray(), y_train)

# Оценка классификатора
predictions = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

print(classification_report(y_test, predictions, ))

KNeighborsClassifier

In [ ]:
# Обучение классификатора
classifier = KNeighborsClassifier()
classifier.fit(X_train.toarray(), y_train)

# Оценка классификатора
predictions = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

print(classification_report(y_test, predictions, ))

Lasso регуляризация (регрессия)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
# прочитайте данные с атрибутами аккаунтов компаний на Facebook и активностью на них
fb = pd.read_csv('/datasets/dataset_facebook_cosmetics.csv', sep = ';')
# разделяем данные на признаки (матрица X) и целевую переменную (y)
X = fb.drop('Total Interactions', axis = 1)
y = fb['Total Interactions']
# разделяем модель на обучающую и валидационную выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# стандартизируем данные методом StandartScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_st = scaler.transform(X_train)
X_test_st = scaler.transform(X_test)
model = Ridge() # создаём модель класса Ridge-регрессия
model.fit(X_train, y_train)
y_pred = model.predict(X_val) 

Random Forest

n_estimators — число деревьев, на основании которых будем строить лес, глубину дерева max_depth, размер подвыборки признаков max_features, минимальное количество объектов в узле min_samples_leaf

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# зададим алгоритм для новой модели на основе алгоритма случайного леса
rf_model = RandomForestRegressor(n_estimators = 100)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_val) 

Градиентный бустинг

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# зададим алгоритм для новой модели на основе алгоритма градиентного бустинга
gb_model = GradientBoostingRegressor(n_estimators = 100)
gb_model.fit(X_train, y_train)
y_pred = gb_model.predict(X_val) 

Логистическя регрессия и установка порога отсечения

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
# Загружаем набор данных Iris
iris = datasets.load_iris()
X = iris.data[iris.target != 2]  # Используем только два класса
y = iris.target[iris.target != 2]
# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Создаем модель логистической регрессии
model = LogisticRegression()
# Обучаем модель
model.fit(X_train, y_train)

In [ ]:
# Предсказываем вероятности для тестовой выборки
probabilities = model.predict_proba(X_test)
# Предсказываем классы на основе стандартного порога 0.5
predictions = model.predict(X_test)
# Выводим вероятности и предсказанные классы
print("Предсказанные вероятности:\n", probabilities)
print("Предсказанные классы:\n", predictions)

В матрице путаницы каждая строка соответствует реальному классу, а каждый столбец — предсказанному классу.  
Следовательно True Positive будут значения на главной диагонали.

In [ ]:
# Оценка точности модели
accuracy = accuracy_score(y_test, predictions)
print("Точность модели:", accuracy)

# Выводим матрицу путаницы и отчет о классификации
print("Матрица путаницы:\n", confusion_matrix(y_test, predictions))
print("Отчет о классификации:\n", classification_report(y_test, predictions))

Если нам нужно установить свой порог (например, у нас классы несбалансированы), то делаем так

In [ ]:
# Установите новый порог
threshold = 0.7
# Предсказание вероятностей для тестовой выборки
probabilities = model.predict_proba(X_test)
# Классификация на основе нового порога
predictions = (probabilities[:, 1] >= threshold).astype(int)
# Выводим предсказанные классы
print("Предсказанные классы с порогом 0.7:\n", predictions)
# Оценка точности модели с новым порогом
accuracy = accuracy_score(y_test, predictions)
print("Точность модели с порогом 0.7:", accuracy)
# Выводим матрицу путаницы и отчет о классификации
print("Матрица путаницы с порогом 0.7:\n", confusion_matrix(y_test, predictions))
print("Отчет о классификации с порогом 0.7:\n", classification_report(y_test, predictions))

Для бинарного классификатора важно постриоть Roc-Auc кривую.

In [ ]:
import numpy as np
import plotly.graph_objects as go
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score

# Загрузка данных
X, y = load_breast_cancer(return_X_y=True)
# Обучение модели
model = LogisticRegression(solver="liblinear", random_state=0)
model.fit(X, y)
# Получение предсказанных вероятностей
y_scores = model.predict_proba(X)[:, 1]
# Вычисление ROC-кривой
fpr, tpr, thresholds = roc_curve(y, y_scores)
# Вычисление AUC
roc_auc = roc_auc_score(y, y_scores)
# Построение ROC-кривой с использованием Plotly
fig = go.Figure()
# Добавляем линию ROC-кривой
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC Curve (AUC = {:.2f})'.format(roc_auc),
                          line=dict(color='blue', width=2)))
# Добавляем линию случайного угадывания
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Guessing',
                          line=dict(color='red', dash='dash')))
# Настройка графика
fig.update_layout(title='Receiver Operating Characteristic (ROC) Curve',
                  xaxis_title='False Positive Rate',
                  yaxis_title='True Positive Rate',
                  xaxis=dict(range=[0, 1]),
                  yaxis=dict(range=[0, 1]),
                  showlegend=True)
# Отображение графика
fig.show()

## Класстеризация

По ссылке можно посмотреть базовые преимущества и недостатки методов кластеризации <br>
https://scikit-learn.org/stable/modules/clustering.html

KMeans

In [9]:
iris_df.data[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [6]:
# Описываем модель
model = KMeans(n_clusters=3)
# Проводим моделирование
model.fit(iris_df.data)
# Предсказание на всем наборе данных
all_predictions = model.predict(iris_df.data)
all_predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1])

### Оценка качества модели

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

# определим функцию, которая будет выводить наши метрики
def print_all_metrics(y_true, y_pred, y_proba, title='Метрики классификации'):
    print(title)
    print('\tAccuracy: {:.2f}'.format(accuracy_score(y_true, y_pred)))
    print('\tPrecision: {:.2f}'.format(precision_score(y_true, y_pred)))
    print('\tRecall: {:.2f}'.format(recall_score(y_true, y_pred)))
    print('\tF1: {:.2f}'.format(f1_score(y_true, y_pred)))
    print('\tROC_AUC: {:.2f}'.format(roc_auc_score(y_true, y_proba)))
